# DecisionTree Assignment - 20기 OOO

물음표 친 부분을 채우고 코드에 대한 주석을 자세하게 달아주세요!

# Data Loading

In [1]:
import pandas as pd 
import numpy as np

from functools import reduce # reduce (누적) 사용

In [2]:
pd_data = pd.read_csv('https://raw.githubusercontent.com/AugustLONG/ML01/master/01decisiontree/AllElectronics.csv')
pd_data.drop("RID",axis=1, inplace = True) #RID는 그냥 순서라서 삭제
pd_data

,age,income,student,credit_rating,class_buys_computer
0,youth,high,no,fair,no
1,youth,high,no,excellent,no
2,middle_aged,high,no,fair,yes
3,senior,medium,no,fair,yes
4,senior,low,yes,fair,yes
5,senior,low,yes,excellent,no
6,middle_aged,low,yes,excellent,yes
7,youth,medium,no,fair,no
8,youth,low,yes,fair,yes
9,senior,medium,yes,fair,yes


## Gini 계수를 구하는 함수 만들기

<img src="gini.png" width="200">

- Input: df(데이터), label(타겟변수명)
- 해당 결과는 아래와 같이 나와야 합니다.

- 지니계수는 데이터의 통계적 분산 정도를 정량화 해서 표현한 값이다.
- 어떤 집합의 gini index가 높을수록 그 집단의 데이터가 분산되어 있음을 확인할 수 있다.

In [3]:
def get_gini(df, label):
    
    # df 데이터의 전체 개수
    data_len = df[label].count()
    
    # 각 클래스별 count를 담은 Generator를 생성
    count_arr = (value for key, value in df[label].value_counts().items())
    
    # reduce lambda를 이용하여 지니 지수 계산
    gini = reduce(lambda x, y: x - (y/data_len)**2, count_arr, 1)
    
    return gini

In [4]:
get_gini(pd_data,'class_buys_computer')

0.4591836734693877

## Feature의 Class를 이진 분류로 만들기
- ex) {A,B,C} -> ({A}, {B,C}), ({B}, {A,C}), ({C}, {A,B})

- Input: df(데이터), attribute(Gini index를 구하고자 하는 변수명)
- Income 변수를 결과로 출력해주세요.

In [5]:
from itertools import combinations

def get_binary_split(df, attribute):
    
    uniques = list(df[attribute].unique()) # 속성 데이터 고유값들을 담은 리스트 
    
    # 결과에 대한 list 생성
    result = []
    
    # 조합(combinations) 이용해서 원소가 1, unique한 원소 개수보다 1개 작은 숫자까지 조합 생성
    for i in range(1, len(uniques)) :
        # 2^(n-1) - 1개 구해짐
        for att in combinations(uniques, i) :
            result.append(list(att))
    
    return result

In [6]:
get_binary_split(pd_data,'income')

[['high'],
 ['medium'],
 ['low'],
 ['high', 'medium'],
 ['high', 'low'],
 ['medium', 'low']]

## 모든 이진분류의 경우의 Gini index를 구하는 함수 만들기
- 위에서 완성한 두 함수를 사용하여 만들어주세요!
- 해당 결과는 아래와 같이 나와야 합니다.

In [7]:
def get_attribute_gini_index(df, attribute, label):
    
    # 결과 담을 dictionary 생성
    result = {}
    
    # 이진분류 생성
    binary_split = get_binary_split(df, attribute)
    
    # 데이터프레임 개수
    n = len(df)
    
    # 이진분류 반복문
    for col in binary_split :
        # dictionary에 담을 이름 선언(2개 이상일 경우 _으로 구분)
        col_name = "_".join(col)
        
        # 반전 연산(~)을 수행하기 위해서 numpy 배열 생성
        # attribute(속성)이 col에 속하면 할당
        mask = np.array(list(map(lambda x : x in col, df[attribute])))

        # 원하는 dataframe만 가져오기
        target = df.loc[mask, :]
        rest = df.loc[~mask, :]
        
        # 지니지수 계산 (t -> target // r -> rest)
        gini_t = get_gini(target, label)
        gini_r = get_gini(rest, label)
        
        # 최종적인 지니지수 계산
        gini = len(target) / n * gini_t + len(rest) / n * gini_r
        
        # 처음에 생성한 dictionary에 추가
        result[col_name] = gini
    return result

In [8]:
get_attribute_gini_index(pd_data, 'income', 'class_buys_computer')

{'high': 0.4428571428571429,
 'medium': 0.4583333333333333,
 'low': 0.45,
 'high_medium': 0.45,
 'high_low': 0.4583333333333333,
 'medium_low': 0.4428571428571429}

- 여기서 가장 작은 Gini index값을 가지는 class를 확인합니다.

In [9]:
min(get_attribute_gini_index(pd_data, 'income', 'class_buys_computer').items())

('high', 0.4428571428571429)

In [10]:
min(get_attribute_gini_index(pd_data, 'income', 'class_buys_computer').items())[0]

'high'

## 분류를 하는 데 가장 중요한 변수를 선정하고, 해당 변수의 Gini index를 제시해주세요.
- 모든 변수에 대한 Gini index(최소)를 출력해주세요.
- 해당 결과는 아래와 같이 나와야 합니다.

In [11]:
# 변수명 중 마지막에 위치한 label 컬럼 얻기
label = pd_data.columns[-1]
# label 변수를 제외한 변수명 얻기
features = list(pd_data.columns[:-1])

# 각 변수를 대상으로 반복문 수행(해당 변수 중 가장 낮은 gini 계수와 변수 출력)
for feature in features:

    # 해당 변수에 대해서 가장 낮은 gini 계수와 변수 출력
    min_gini = min(get_attribute_gini_index(pd_data, feature, label).items())[1]
    print(f"Minimum Gini index of {feature} : {min_gini :.4f}")
    

Minimum Gini index of age : 0.3571
Minimum Gini index of income : 0.4429
Minimum Gini index of student : 0.3673
Minimum Gini index of credit_rating : 0.4286


gini index가 가장 작게 나온 'age'를 가장 중요한 변수로 선정합니다.

이어서 해당 변수의 이진 분류된 각 class에 대해 Gini index도 계산합니다.

In [12]:
get_attribute_gini_index(pd_data, 'age', 'class_buys_computer')

{'youth': 0.3936507936507937,
 'middle_aged': 0.35714285714285715,
 'senior': 0.4571428571428572,
 'youth_middle_aged': 0.4571428571428572,
 'youth_senior': 0.35714285714285715,
 'middle_aged_senior': 0.3936507936507937}

'age' 변수에서 gini index가 가장 작게 나온 'middle_aged' class를 선정합니다.

## Entropy 를 구하는 함수 만들기

<img src = https://miro.medium.com/max/1122/0*DkWdyGidNSfdT1Nu.png width = "350">

In [13]:
from math import log2

def getEntropy(df, feature) :
    
    
    """
    수식을 참고하여,
    데이터프레임 df에서 특정 feature에 대해 
    엔트로피를 구하는 함수를 작성해주세요.
    """
    
    # 데이터 전체 길이 할당
    data_len = df[feature].count()
    
    # reduce 함수를 이용하여 초기값 0에 대해서 각 feature 별 엔트로피 식에 대입한 값 누적으로 더하기
    return reduce(lambda x, y : x + (-(y[1]/data_len) * np.log2(y[1]/data_len)), df[feature].value_counts().items(), 0)

In [14]:
getEntropy(pd_data, "class_buys_computer")

0.9402859586706311

In [15]:
# 가장 중요한 변수로 선정된 목표변수를 제외한 다른 변수들에 대해
# 각 칼럼별로 엔트로피를 구해주는 함수를 작성해주세요.

def getGainA(df, feature) :
    
    # 결과값에 대한 초기 dictionary 생성    
    result = {}

    # 목표변수 feature에 대한 엔트로피 구하기
    info_D = getEntropy(df, feature)
    # 목표변수를 제외한 나머지 변수들 리스트 형태로 할당
    columns = list(df.loc[: , df.columns != feature])
    
    for col in columns : 
        # 가중치 초기화
        w = 0
        
        # 각 특성에 대한 고유값 반복
        for i in list(df[col].unique()) : 
            # 전체 데이터 개수 구하기
            data_len = len(df[feature])
            # 특성 col의 값이 i인 데이터 개수 구하기 (-> 가중치)
            data_len_i = sum(df[col] == i)
            
            # 가중치와 해당 특성 값에 대한 엔트로피의 곱을 누적하여 가중치 계산
            w += (data_len_i / data_len) * getEntropy(df[df[col] == i], feature)
        
        # col에 대한 정보 획득량 계산하여 저장 -> 정보의 양
        result[col] = info_D - w
    
    return(result)

In [16]:
getGainA(pd_data, "class_buys_computer")

{'age': 0.24674981977443933,
 'income': 0.02922256565895487,
 'student': 0.15183550136234159,
 'credit_rating': 0.04812703040826949}

#### `정보 획득량이 가장 높은 변수는 age이다.`